In [ ]:
from google.colab import files
files.upload()  # Upload the kaggle.json file here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sriramakula212","key":"00d24e3ca44f844d63057892e1efef45"}'}

In [ ]:
import os
import zipfile

# Create a Kaggle directory and move kaggle.json there
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
# Replace with the actual dataset name from Kaggle
!kaggle datasets download -d meetnagadia/human-action-recognition-har-dataset


Dataset URL: https://www.kaggle.com/datasets/meetnagadia/human-action-recognition-har-dataset
License(s): ODbL-1.0
human-action-recognition-har-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Unzip the downloaded dataset
!unzip -q human-action-recognition-har-dataset.zip


replace Human Action Recognition/Testing_set.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Human Action Recognition/Training_set.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace Human Action Recognition/test/Image_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace Human Action Recognition/test/Image_10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace Human Action Recognition/test/Image_100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Human Action Recognition/test/Image_100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A


In [ ]:
# List extracted folders to find correct paths
os.listdir()


['.config',
 'Human Action Recognition',
 'human-action-recognition-har-dataset.zip',
 'sample_data']

In [ ]:
train_path = "/content/Human Action Recognition/train"
test_path = "/content/Human Action Recognition/test"
train_csv_path = "/content/Human Action Recognition/Training_set.csv"
test_csv_path = "/content/Human Action Recognition/Testing_set.csv"


In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)


In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    class_mode='raw',  # use 'categorical' if one-hot, 'raw' if integers
    batch_size=32,
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    class_mode='raw',
    batch_size=32,
    shuffle=False
)


Found 10080 validated image filenames.
Found 2520 validated image filenames.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
base_model.trainable = True
model.compile(optimizer=Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.0898 - loss: 3.6966

315/315 ━━━━━━━━━━━━━━━━━━━━ 246s 566ms/step - accuracy: 0.0899 - loss: 3.6954 - val_accuracy: 0.3766 - val_loss: 2.0243 - learning_rate: 1.0000e-05
Epoch 2/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.2150 - loss: 2.5683

315/315 ━━━━━━━━━━━━━━━━━━━━ 168s 533ms/step - accuracy: 0.2151 - loss: 2.5679 - val_accuracy: 0.5020 - val_loss: 1.6273 - learning_rate: 1.0000e-05
Epoch 3/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - accuracy: 0.3561 - loss: 2.0448

315/315 ━━━━━━━━━━━━━━━━━━━━ 172s 546ms/step - accuracy: 0.3562 - loss: 2.0445 - val_accuracy: 0.5873 - val_loss: 1.3777 - learning_rate: 1.0000e-05
Epoch 4/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.4575 - loss: 1.7288

...

315/315 ━━━━━━━━━━━━━━━━━━━━ 174s 551ms/step - accuracy: 0.7807 - loss: 0.7474 - val_accuracy: 0.7671 - val_loss: 0.8063 - learning_rate: 1.0000e-05
Epoch 14/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - accuracy: 0.7849 - loss: 0.7057

315/315 ━━━━━━━━━━━━━━━━━━━━ 199s 543ms/step - accuracy: 0.7849 - loss: 0.7057 - val_accuracy: 0.7730 - val_loss: 0.7989 - learning_rate: 1.0000e-05
Epoch 15/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 166s 526ms/step - accuracy: 0.8023 - loss: 0.6635 - val_accuracy: 0.7706 - val_loss: 0.8193 - learning_rate: 1.0000e-05
Epoch 16/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 503ms/step - accuracy: 0.8114 - loss: 0.6313

315/315 ━━━━━━━━━━━━━━━━━━━━ 170s 538ms/step - accuracy: 0.8115 - loss: 0.6312 - val_accuracy: 0.7790 - val_loss: 0.8086 - learning_rate: 1.0000e-05
Epoch 17/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 505ms/step - accuracy: 0.8252 - loss: 0.5749

315/315 ━━━━━━━━━━━━━━━━━━━━ 172s 545ms/step - accuracy: 0.8252 - loss: 0.5750 - val_accuracy: 0.7813 - val_loss: 0.8224 - learning_rate: 1.0000e-05
Epoch 18/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.8356 - loss: 0.5480

315/315 ━━━━━━━━━━━━━━━━━━━━ 201s 542ms/step - accuracy: 0.8356 - loss: 0.5480 - val_accuracy: 0.7853 - val_loss: 0.8031 - learning_rate: 2.0000e-06
Epoch 19/30
315/315 ━━━━━━━━━━━━━━━━━━━━ 164s 521ms/step - accuracy: 0.8391 - loss: 0.5217 - val_accuracy: 0.7849 - val_loss: 0.8020 - learning_rate: 2.0000e-06


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    target_size=(224, 224),
    class_mode=None,  # No labels
    batch_size=32,
    shuffle=False
)


Found 5400 validated image filenames.


In [ ]:
model.load_weights("best_model.h5")  # Load best weights

# Evaluate on validation
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc:.2f}")

# Predict on test if needed
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    target_size=(224, 224),
    class_mode=None,
    batch_size=32,
    shuffle=False
)
predictions = model.predict(test_generator)


79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - accuracy: 0.7906 - loss: 0.7788
Validation Accuracy: 0.79
Found 5400 validated image filenames.
169/169 ━━━━━━━━━━━━━━━━━━━━ 23s 113ms/step
